## 웨더아이 날씨 데이터 크롤링
- 2019.01~2020.08
- 서울, 파주, 수원, 울산, 부산, 광주, 대전, 대구, 강릉

In [129]:
import requests
import openpyxl
from bs4 import BeautifulSoup
import os
import csv
import pprint
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from selenium import webdriver
import time
import re

In [132]:
# 매 파일마다의 칼럼(수집을 원하는 데이터)
columns = ["날짜","평균기온","최고기온","최저기온","강수량(mm)","신적설(cm)","평균풍속(m/s)","평균습도(%)","운량(1/10)","일조시간(Hr)","날씨"]

# 수집을 원하는 지역들
select_places = ["서울","파주","수원","울산","부산","광주","대전","대구","강릉"] 

In [222]:
driver=webdriver.Chrome("./chromedriver") 

driver.get("https://www.weatheri.co.kr/bygone/bygone01.php")
time.sleep(3)

for place_idx in range(96):
    """
    main frame(검색 옵션 지정 후 검색)
    """
    # 지역, 년도 선택창 -> 매 검색마다 업데이트 되기 때문에 한번 검색 후 다시 새롭게 element 읽어옴
    select = driver.find_elements_by_css_selector("select")
    place_select = select[0]
    year_select = select[1]
    places = place_select.find_elements_by_css_selector("option") # 후보 지역들
    
    p = places[place_idx].text # 지역 이름 추출
    
    # 원하는 지역이 아니면 넘어감
    if p not in select_places:
        continue
    
    # 원하는 지역이 맞다면 옵션으로 선택
    places[place_idx].click()
    
    # 지역 날씨 파일 생성
    wb=openpyxl.Workbook()
    sheet=wb.active
    sheet.append(columns)
    
    year_select.find_elements_by_css_selector("option")[1].click() # 2019년으로 시작 년도 변경
    driver.find_elements_by_css_selector("tbody > tr > td img")[15].click() # 검색
    time.sleep(2)
    
    # 월별 데이터 수집
    driver.switch_to.frame("kako") # 데이터가 들어있는 iframe 페이지로 변환
    
    """
    iframe(데이터 수집)
    """
    for m in range(20):
        # 월 검색 옵션 -> 매 검색마다 새롭게 업데이트 되기 때문에 element 새롭게 읽어옴
        months_select = driver.find_element_by_css_selector("select") # 월선택창 클릭
        months_select.click()
    
        months = months_select.find_elements_by_css_selector("option") # 월 선택
        months[m].click()
        
        # 월별 데이터 보기
        driver.find_elements_by_css_selector("td > img")[-1].click() # 보기 클릭
        time.sleep(1)
        
        # 데이터 수집
        # 한 날짜(한 행)
        container = driver.find_elements_by_css_selector("tbody tbody tbody tr")[1:] # 칼럼 행 제거
        
        for c in container:
            data = c.find_elements_by_css_selector("td") # 한 날짜의 모든 데이터
            d = data[0].text[:-4] # 날짜 데이터의 공백 및 요일 제거
            d.replace("\n","")
            d=re.findall(r"\d+",d) # 숫자만 읽어옴
            d="-".join(d) # date 형식으로 변환
            one_row = [d]
            
            # 해당 날짜의 날씨 데이터
            for i in range(1,11):
                one_row.append(data[i].text) 
                
            sheet.append(one_row) # sheet에 추가
  
    wb.save(f"{p}날씨.xlsx") # 해당 지역 2019~2020 날씨 문서 저장
    
    driver.switch_to.default_content() # 다시 상위 페이지로 변환